In [1]:
import re
import pickle
from heapq import nlargest
import time

import pandas as pd
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import session_info

In [2]:
# Посмотрим версии используемых модулей.
session_info.show()

In [3]:
# Загрузим ранее сохранённые уникальные навазния компаний.
text_unic = pickle.load(open('companies_name_unic.pickle','rb'))

# Загрузим ранее сохранённый tfidfvectorizer.
my_vectorizer = pickle.load(open('vectorizer_2.pickle','rb'))

def sentence_embedding_fl(string, vectorizer):
    tf_vectorizer = vectorizer
    sentence_list = [tf_vectorizer.transform([word]).toarray() for word in string.split()]
        
    result = np.sum(np.array(sentence_list), axis=0)
    if len(result.shape) == 0:
        result = np.zeros((1,9776))
    
    return result

In [4]:
# Загрузим ранее сохранённые эмбеддинги для названий компаний.
name_embeddings = pickle.load(open('name_embeddings.pickle','rb'))

In [5]:
# Топ для вывода и порог для косинусного сходства.
top = 10
treshold_for_cosine_similarity = 0.5

# Функция для поиска топ 10 схожих названий и их вывода.
def find_top(text, top, treshold):
    
    # Получим эмбеддинг текста
    text_vector = sentence_embedding_fl(text, my_vectorizer)
    
    # Вычислим косинусное сходство 
    cosine_similarity_list = [cosine_similarity( text_vector, sentence_vector)[0] for sentence_vector in name_embeddings ]
    
    # Ищем схожие названия
    company_names = []
    maximums = nlargest(top, cosine_similarity_list)
    maximums_treshold = [max for max in maximums if max >= treshold ]
    if len(maximums_treshold) > 0:
            indexes_set = set()
            for maximum in maximums_treshold:
                indexes = list(np.where(np.array(cosine_similarity_list) == maximum)[0])
                for index in indexes:
                    if index not in indexes_set:
                        indexes_set.add(index)
                        company_names.append(text_unic[index])

    if len(company_names) < top:
        result = company_names
    else:
        result = company_names[:top]
        
    
    print("Косинусное сходство ", [round(float(element),3) for element in maximums])
    print()

    if len(result) > 0:
        for name in result:
            print(name)
    else:
        print('Похожих названий, превышающих порог ',treshold, " не обнаружено." )
        
        
        
# Второй вариант функции, добавленный для работы с функцией main()        
def find_top_2(text, top, treshold):
    
    # Получим эмбеддинг текста
    text_vector = sentence_embedding_fl(text, my_vectorizer)
    
    # Вычислим косинусное сходство 
    cosine_similarity_list = [cosine_similarity( text_vector, sentence_vector)[0] for sentence_vector in name_embeddings ]
    
    # Ищем схожие названия
    company_names = []
    maximums = nlargest(top, cosine_similarity_list)
    maximums_treshold = [max for max in maximums if max >= treshold ]
    if len(maximums_treshold) > 0:
            indexes_set = set()
            for maximum in maximums_treshold:
                indexes = list(np.where(np.array(cosine_similarity_list) == maximum)[0])
                for index in indexes:
                    if index not in indexes_set:
                        indexes_set.add(index)
                        company_names.append(text_unic[index])

    if len(company_names) < top:
        result = company_names
    else:
        result = company_names[:top]
    
    return result

In [6]:
# Время начала обработки поиска. 
start_time = time.time()

# Тестовый текст 1. Сначала возьмём порог 0.5 для косинусного сходства.
my_test_text_1 = 'Darth Vader is a great Sith Lord'

# Найдем схожие названия
find_top(my_test_text_1, top, treshold_for_cosine_similarity)

# Время окончания обработки поиска.
end_time = time.time()

print()
print("Время обработки поиска = ", round(end_time-start_time, 3))

Косинусное сходство  [0.447, 0.447, 0.447, 0.355, 0.316, 0.316, 0.316, 0.316, 0.316, 0.316]

Похожих названий, превышающих порог  0.5  не обнаружено.

Время обработки поиска =  11.204


In [7]:
# Видим, что ничего не найдено, так косинусное сходство меньше порога.

In [8]:
# Время начала обработки поиска. 
start_time = time.time()

treshold_for_cosine_similarity = 0.3

# Тестовый текст 1. Теперь будем использовать порог 0.3.
my_test_text_1 = 'Darth Vader is a great Sith Lord'

find_top(my_test_text_1, top, treshold_for_cosine_similarity)

# Время окончания обработки поиска.
end_time = time.time()

print()
print("Время обработки поиска = ", round(end_time-start_time, 3))

Косинусное сходство  [0.447, 0.447, 0.447, 0.355, 0.316, 0.316, 0.316, 0.316, 0.316, 0.316]

Lord Industrial Ltda
Lord India Pvt., Ltd.
Jm Lord International Llc
Greatech Philippines Inc.
Great Sports Infra Pvt., Ltd.
Shenzhen Great Machinery Co., Ltd.
Great Worldwide Logistics
Great Luck Inc.
Profit Great Inc.
Great Champ

Время обработки поиска =  11.728


In [9]:
# Видим, что к непохожему на название компании тексту нашлись названия, где фигурируют похожие слова "Lord" и "Great".

In [10]:
# Время начала обработки поиска. 
start_time = time.time()

# Теперь попробуем текст, который был в датасете.
my_test_text_2 = 'Powermax Rubber Factory'

find_top(my_test_text_2, top, treshold_for_cosine_similarity)

# Время окончания обработки поиска.
end_time = time.time()

print()
print("Время обработки поиска = ", round(end_time-start_time, 3))

Косинусное сходство  [0.816, 0.577, 0.577, 0.577, 0.577, 0.438, 0.428, 0.412, 0.412, 0.412]

Powermax Rubber Factory
B G N Rubber Factory
Western Rubbers India Pvt., Ltd.
K.S. Rubbers
Imp. Rubbers International Llc
Rubberchem (Pty) ) Ltd.
N.D.Rubber
Power Trade Corporation
D J Powers Co Inc.
Ge Power India

Время обработки поиска =  11.127


In [11]:
# Видим что на первом месте исходный текст (не удивительно), а дальше тексты с похожими словами. 

In [12]:
# Время начала обработки поиска. 
start_time = time.time()

# Теперь попробуем текст, не был в датасете, но похож, на то, что был там.
# Оригинал: Honeywell Aerospace Systems Laboratory Co. S. De .R.L. De C.V.
# Изменённый вариант
my_test_text_3 = 'Well Aero System Lab Industrial Craft'

find_top(my_test_text_3, top, treshold_for_cosine_similarity)

# Время окончания обработки поиска.
end_time = time.time()

print()
print("Время обработки поиска = ", round(end_time-start_time, 3))

Косинусное сходство  [0.494, 0.45, 0.408, 0.408, 0.408, 0.408, 0.408, 0.408, 0.408, 0.382]

Honeywell Aerospace Systems Laboratory Co. S. De .R.L. De C.V.
Aerocosta Global Systems Inc.
Craft Colombia Sas
Mx Systems International P Ltd.
Craft Argentina S.A.
Aero Rubber Co. Inc.
Systems Group Llc
Wells
Sx Well Usa Llc
M Industria S.A De C.V.

Время обработки поиска =  13.346


In [13]:
# Видим что на первом месте текст, который был изменён а дальше тексты с похожими словами. 

In [14]:
# Время поиска и вывода похожих названий около 11 секунд. Долго потому что каждый раз поиск запускается для 18022 эмбеддингов.

In [21]:
# Функция позволит вводить строку для поиска с клавиатуры.
def main():
    while True:
        comp_name = input("Введите название компании:\n\n").rstrip()
        if comp_name == "exit":
            break
        k = 10
        
        start_time = time.time()
        
        top_comp = find_top_2(comp_name, k, treshold_for_cosine_similarity) 
        
        if len(top_comp) > 0:    
            print (f"Топ {k} похожих компаний:\n\n")
            for i, comp in enumerate(top_comp):
                print (f"{i + 1}: {comp}")
        
        else:
            print ("Похожих компаний нет в списке\n\n")
        print("\n")
        end_time = time.time()

        print("Время обработки поиска = ", round(end_time-start_time, 3))
        print()

In [22]:
# После ввода текста приходится ждать около 10 секунд.
main()

Введите название компании:

Honeywell Aerospace Systems Laboratory Co. S. De .R.L. De C.V.
Топ 10 похожих компаний:


1: Honeywell Aerospace Systems Laboratory Co. S. De .R.L. De C.V.
2: Honeywell Aerospace De Mexico Sa De Cv
3: TA Aerospace
4: Aerospace B V
5: Honeywell International Inc.
6: Mx Systems International P Ltd.
7: Systems Group Llc
8: Aerocosta Global Systems Inc.
9: Oral B Laboratories
10: Winsystems Corporation


Время обработки поиска =  11.253

Введите название компании:

Powermax Rubber Factory
Топ 10 похожих компаний:


1: Powermax Rubber Factory
2: B G N Rubber Factory
3: Western Rubbers India Pvt., Ltd.
4: K.S. Rubbers
5: Imp. Rubbers International Llc
6: Rubberchem (Pty) ) Ltd.
7: N.D.Rubber
8: Power Trade Corporation
9: D J Powers Co Inc.
10: Ge Power India


Время обработки поиска =  11.2

Введите название компании:

hfgh 547474 fsjthu55
Топ 10 похожих компаний:


1: Thuan Loi Rubber
2: Thuan Loi Rubber Co., Ltd.


Время обработки поиска =  11.155

Введите назва

In [23]:
# Загрузим датасет.
df = pd.read_csv('train.csv')

In [24]:
top = 10

# Возьмём из датасета те названия, что имеют схожие.
df_duplicate = df[df['is_duplicate'] == 1]

name_1_duplicate = list(df_duplicate['name_1'])
name_2_duplicate = list(df_duplicate['name_2'])

# Создадим словарь, чтобы можно было найти дубль для каждого названия из списка дубликатов.
duplicate_dictionary = dict(zip(name_1_duplicate+name_2_duplicate, name_2_duplicate+ name_1_duplicate))


# Функция подсчёта метрики ранжирования.
def search_acc_metric(names):
    d_s = []
    for name in names:
        try:
            duplicate_dictionary[name]
            d_s.append(1)
        except KeyError:
            d_s.append(0)
    
    k = len(names)
    if 1 in d_s:
        w_i = [max(-0.2*i+1,0) for i in range(k)]
        result = min(sum([x*y for x, y in zip(w_i, d_s)]), 1)
    else:
        w_i = [0.2*i for i in range(1, k+1)]
        result = max(1 - sum(w_i), 0)

    return result

In [25]:
df_duplicate.head(10)

,pair_id,name_1,name_2,is_duplicate
161,162,JX Nippon Oil & Gas Exploration (Brasil) Ltda,JX Nippon Oil & Gas Exploration Technical Serv...,1
603,604,Pirelli Neumaticos S.A.I.C.,"Pirelli Tyre Co., Ltd.",1
835,836,Brenntag Australia (Pty) Ltd.,Brenntag Group,1
1328,1329,"PAUL BAUDER GMBH & CO KG, BOCHUM PLANT",Paul Bauder ag,1
1562,1563,TOTAL CESKA REPUBLIKA s.r.o.,TOTAL FRANCE (ARNAY LE DUC),1
1786,1787,Pt Bridgestone Tire Indonesia,Bridgestone Do Brasil Industria E Comenrcio Ltda,1
1798,1799,Brenntag Peru S.A.C.,Brenntag Group,1
1962,1963,"Carlisle Coatings & Waterproofing, Inc.",Carlisle Coatings & Wtrprfng,1
2054,2055,Pt Bridgestone Tire Indonesia,"Thai Bridgestone Co., Ltd.",1
2772,2773,Onduline Polska Sp. z o.o.,Onduline S.A.,1


In [26]:
# Возьмём название у которого есть похожее и посмотрим значение метрики.
companies_names = find_top_2("Carlisle Coatings & Waterproofing, Inc.", top, treshold_for_cosine_similarity)

for companies in companies_names:
    print(companies)

Carlisle Coatings & Waterproofing, Inc.
CARLISLE COATINGS
Carlisle Coatings & Wtrprfng
I Do Roofing Inc.
Imperial Water Proofing Industries Pvt., Ltd.
Bitufa Waterproofing B.V.
Coats Mexico S.A. De C.V.
Coats Mexico S.A De C.V
Coats Honduras Sa
Coats Mexico Sa De Cv


In [27]:
# Теперь посчитаем метрику   
print(search_acc_metric(companies_names))

1


In [28]:
# Возьмём название у которого нет похожего и посмотрим значение метрики.
companies_names = find_top_2("Powermax Rubber Factory", top, treshold_for_cosine_similarity)

for companies in companies_names:
    print(companies)

Powermax Rubber Factory
B G N Rubber Factory
Western Rubbers India Pvt., Ltd.
K.S. Rubbers
Imp. Rubbers International Llc
Rubberchem (Pty) ) Ltd.
N.D.Rubber
Power Trade Corporation
D J Powers Co Inc.
Ge Power India


In [29]:
# Теперь посчитаем метрику   
print(search_acc_metric(companies_names))

0
